In [6]:
import numpy as np
import pandas as pd
import re

In [7]:
PATH_tweets = '../Data/tweets.csv'

In [8]:
tweets = pd.read_csv(
    PATH_tweets,
    usecols=['id', 'screen_name', 'text'], 
    index_col='id', 
    dtype={'screen_name': str, 'text': str}
    )
tweets = tweets.drop_duplicates(subset='text', keep=False)
tweets = tweets.dropna()

In [11]:
def limpiar_retweet(tweet:str):
    if tweet.startswith('RT'):
        tweet = ''.join(tweet.split(': ')[1:])
    return tweet

def limpiar_hashtag(tweet:str):
    tweet = re.sub(r'#\w+', '', tweet)
    return tweet

def limpiar_url(tweet:str):
    tweet = re.sub(r'http\S+', '', tweet)
    return tweet

def limpiar_emoji(tweet:str):
    tweet = re.sub(r'\\x\w+', '', tweet)
    return tweet

def limpiar_puntuacion(tweet:str):
    tweet = re.sub(r'[^\w\s@]', '', tweet)
    return tweet

def limpiar_espacios(tweet:str):
    tweet = re.sub(r'\s+', ' ', tweet)
    return tweet

def limpiar_mayusculas(tweet:str):
    tweet = tweet.lower()
    return tweet

def remplazar_tildes(tweet:str):
    tweet = tweet.replace('á', 'a')
    tweet = tweet.replace('é', 'e')
    tweet = tweet.replace('í', 'i')
    tweet = tweet.replace('ó', 'o')
    tweet = tweet.replace('ú', 'u')
    return tweet

def limpiar_texto(tweet:str):
    tweet = limpiar_retweet(tweet)
    tweet = limpiar_hashtag(tweet)
    tweet = limpiar_url(tweet)
    tweet = limpiar_emoji(tweet)
    tweet = limpiar_puntuacion(tweet)
    tweet = limpiar_espacios(tweet)
    tweet = limpiar_mayusculas(tweet)
    tweet = remplazar_tildes(tweet)
    tweet = tweet.strip()
    return tweet

tweets['text'] = tweets.loc[:, 'text'].apply(limpiar_texto)

In [12]:
# Save tweets
tweets.to_parquet('../Data/tweets.parquet')